# HOLA AMIGOS

### Imports

In [1]:
% reset
from sklearn.model_selection import train_test_split
import numpy as np
from helpers import *
import os
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pathlib
from multiprocessing import Process,Lock,Value
from gensim.corpora import Dictionary
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
import ntpath
%load_ext autoreload
%autoreload 2

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
ntpath.basename("a/b/c/d/e")

def path_leaf(path):
    a,b,c,d,e = path.split("/")
    return d,e

In [3]:
dirs_train = ['Data/aclImdb/train/pos','Data/aclImdb/train/neg']
clean_root_train = speedy(dirs_train,15)
dirs_test = ['Data/aclImdb/test/pos','Data/aclImdb/test/neg']
clean_root_test = speedy(dirs_test,15)

In [4]:
from pathlib import Path
class MyDocs(object):
    def __init__(self, dirname):
        self.dirname = dirname
        self.pos = []
        self.neg = []
        self.loopy()
 
    def __iter__(self):
        self.loopy()
    
    def loopy(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                f=open(path_in_str)
                parent, uid = path_leaf(path_in_str)
                uid, _ = uid.split("_")
                uid = int(str(uid))
                if parent == "pos":
                    self.pos.append(f.read())
                elif parent == "neg":
                    self.neg.append(f.read())

In [5]:
docs_train = MyDocs(clean_root_train)
docs_test = MyDocs(clean_root_test)
# bigram_transformer = Phrases(docs)

In [7]:
from sklearn.linear_model import SGDClassifier,LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB

pos_train = np.array(docs_train.pos)
neg_train = np.array(docs_train.neg)
X_train = np.concatenate((pos_train,neg_train),axis=0)
y_train = np.asarray([1]*pos_train.shape[0] + [-1]*neg_train.shape[0])

pos_test = np.array(docs_test.pos)
neg_test = np.array(docs_test.neg)
X_test = np.concatenate((pos_test,neg_test),axis=0)
y_test = np.asarray([1]*pos_test.shape[0] + [-1]*neg_test.shape[0])

# X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.1, random_state=42)

### Classifiers

In [8]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),])

In [15]:
text_clf.fit(X_train, y_train)
np.mean(text_clf.predict(X_test) == y_test)

0.85274000000000005

In [20]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

In [21]:
text_clf.fit(X_train, y_train)
np.mean(text_clf.predict(X_test) == y_test)

0.89966000000000002

In [22]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', LogisticRegressionCV(solver='lbfgs')),])

In [23]:
text_clf.fit(X_train, y_train)
np.mean(text_clf.predict(X_test) == y_test)

0.95098000000000005